In [ ]:
import polars as pl
pl.Config(tbl_width_chars=500)
pl.Config(tbl_cols=-1)
import rtsvg
rt = rtsvg.RACETrack()
from  polars_spring_layout import PolarsSpringLayout
import random
import copy
import time

edges_filename  = '../../data/stanford/facebook/348.edges'
_lu_  = {'fm':[], 'to':[]}
_pos_ = {}
for _edge_ in open(edges_filename, 'rt').read().split('\n'):
    if _edge_ == '': continue
    _split_     = _edge_.split()
    _fm_, _to_  = int(_split_[0]), int(_split_[1])
    _lu_['fm'].append(_fm_), _lu_['to'].append(_to_)
    _pos_[_fm_] = [10.0+random.random()*100.0, 20.0+random.random()*100.0]
    _pos_[_to_] = [10.0+random.random()*100.0, 20.0+random.random()*100.0]

#_lu_  = {'fm':'a b c a b c c c c c c c'.split(), 'to':'b c a a0 b0 c0 c1 c2 c3 c4 c5 c6'.split()}
_df_  = pl.DataFrame(_lu_)
_g_   = rt.createNetworkXGraph(_df_, [('fm','to')])
_pos_ = {}
for _node_ in _g_.nodes(): _pos_[_node_] = [random.random(), random.random()]
_iterations_ = None
t0 = time.time()
_pos_ref_ = rt.springLayout   (_g_, copy.deepcopy(_pos_), iterations=_iterations_)
t1 = time.time()
_psl_     = PolarsSpringLayout(_g_, copy.deepcopy(_pos_), iterations=_iterations_)
t2 = time.time()
_pos_     = _psl_.results()
rt.tile([_psl_.svgAnimation(), rt.link(_df_, [('fm','to')], _pos_ref_), rt.link(_df_, [('fm','to')], _pos_)], spacer=10)

In [ ]:
# Ref Time = 5.1s | Polars Time = 0.8s | M1 Pro @ 16GB
print(f'Ref Time = {t1-t0}, Polars Time = {t2-t1}')